In [13]:
# this set of code gets 9 Vanguard Funds closing prices from 1-1-2-15 to 12-31-2019. (This treatment ignores dividends.)
# the dataframe that this creates is called comparison_df.

! pip install alpaca-trade-api

# Load .env enviroment variables

import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline

from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("APCA_API_KEY")
alpaca_secret_key = os.getenv("APCA_SECRET_KEY")

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# # Subset your tickers, then pull returns data:
# # Set the ticker
ticker = ["VTI", "VXUS", "BND", "BNDX", "VUG", "VB", "VOO", "VONG", "VYM"]

# # Set timeframe to '1D'
timeframe = "1D"

# # Set start and end datetimes of 1 year, between now and 365 days ago.
start_date = pd.Timestamp('2015-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2019-12-31', tz='America/New_York').isoformat()


comparison_df = api.get_barset(
     ticker,
     timeframe,
     limit=None,
     start=start_date,
     end=end_date,
     after=None,
     until=None,
 ).df

# # Drop unnecessary columns
comparison_df = comparison_df.drop(columns=['open', 'high', 'low', 'volume'], level=1)

# # Display sample data

comparison_df.head()



,BND,BNDX,VB,VONG,VOO,VTI,VUG,VXUS,VYM
,close,close,close,close,close,close,close,close,close
time,,,,,,,,,
2015-01-02 00:00:00-05:00,82.66,53.2551,116.30,98.0100,188.39,105.93,104.24,48.21,68.7300
2015-01-05 00:00:00-05:00,82.90,53.3300,114.46,96.5200,185.06,104.11,102.43,47.18,67.5000
2015-01-06 00:00:00-05:00,83.14,53.4435,112.99,95.4300,183.26,103.08,101.44,46.79,67.0593
2015-01-07 00:00:00-05:00,83.21,53.4100,114.37,96.5845,185.53,104.31,102.73,47.34,67.8100
2015-01-08 00:00:00-05:00,83.04,53.4000,116.21,98.3900,188.85,106.12,104.54,47.98,69.0900


In [16]:
# the comparision_df stops at the end of 2019 because the real estate data does, this set of code creates a new df
# called monte_carlo_df that takes the last five years of data that's available to make the monte carlo model more accurate.

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# # Subset your tickers, then pull returns data:
# # Set the ticker
ticker = ["VTI", "VXUS", "BND", "BNDX", "VUG", "VB", "VOO", "VONG", "VYM"]

# # Set timeframe to '1D'
timeframe = "1D"

# # Set start and end datetimes of 5 years, between now and 5 years ago.
end_date = pd.Timestamp.today()
start_date = end_date - pd.Timedelta('1825 days')

monte_carlo_df = api.get_barset(
     ticker,
     timeframe,
     limit=None,
     start=start_date,
     end=end_date,
     after=None,
     until=None,
 ).df

# # Drop unnecessary columns
monte_carlo_df = monte_carlo_df.drop(columns=['open', 'high', 'low', 'volume'], level=1)

# # Display sample data

monte_carlo_df.head()



,BND,BNDX,VB,VONG,VOO,VTI,VUG,VXUS,VYM
,close,close,close,close,close,close,close,close,close
time,,,,,,,,,
2020-03-23 00:00:00-04:00,83.970,55.450,98.57,135.3300,204.45,111.74,137.36,36.80,61.0300
2020-03-24 00:00:00-04:00,84.085,55.450,108.22,147.5400,223.82,122.51,150.06,39.88,66.2400
2020-03-25 00:00:00-04:00,85.235,56.000,111.66,148.5400,226.77,124.22,151.04,41.18,67.0604
2020-03-26 00:00:00-04:00,85.630,56.315,118.05,157.4871,240.24,130.97,159.15,42.95,71.6500
2020-03-27 00:00:00-04:00,86.030,56.470,114.46,151.7100,232.86,126.90,153.76,41.39,69.7700


In [ ]:
# now that we have a data frame with the last five years closing prices this set of code predicts prices for the 
# next five years.

# Calculate the daily roi for the nine index funds.

daily_returns = monte_carlo_df.pct_change()

# Display sample data

daily_returns.head()

# Calculate average daily returns. 

avg_daily_return_bnd = daily_returns.mean()['BND']['close']
avg_daily_return_bndx = daily_returns.mean()['BNDX']['close']
avg_daily_return_vb = daily_returns.mean()['VB']['close']
avg_daily_return_vong = daily_returns.mean()['VONG']['close']
avg_daily_return_voo = daily_returns.mean()['VOO']['close']
avg_daily_return_vti = daily_returns.mean()['VTI']['close']
avg_daily_return_vug = daily_returns.mean()['VUG']['close']
avg_daily_return_vxus = daily_returns.mean()['VXUS']['close']
avg_daily_return_vtm = daily_returns.mean()['VTM']['close']



# Compute daily volatility. the #'d out code prints the results.

std_dev_return_bnd = daily_returns.std()['BND']['close']
std_dev_return_bndx = daily_returns.std()['BNDX']['close']
std_dev_return_vb = daily_returns.std()['VB']['close']
std_dev_return_vong = daily_returns.std()['VONG']['close']
std_dev_return_voo = daily_returns.std()['VOO']['close']
std_dev_return_vti = daily_returns.std()['VTI']['close']
std_dev_return_vug = daily_returns.std()['VUG']['close']
std_dev_return_vxus = daily_returns.std()['VXUS']['close']
std_dev_return_vtm = daily_returns.std()['VTM']['close']

# Set the last price to being the Monte Carlo simulation.

bnd_last_price = monte_carlo_df['BND']['close'][-1]
bndx_last_price = monte_carlo_df['BNDX']['close'][-1]
vb_last_price = monte_carlo_df['VB']['close'][-1]
vong_last_price = monte_carlo_df['VONG']['close'][-1]
voo_last_price = monte_carlo_df['VOO']['close'][-1]
vti_last_price = monte_carlo_df['VTI']['close'][-1]
vug_last_price = monte_carlo_df['VUG']['close'][-1]
vxus_last_price = monte_carlo_df['VXUS']['close'][-1]
vtm_last_price = monte_carlo_df['VTM']['close'][-1]

# Setup the Monte Carlo Parameters
number_simulations = 500
number_records = 252 * 30
monte_carlo = pd.DataFrame()

# Run the Monte Carlo Simulation

# PICK UP REVISING CODE HERE - THE CODE BELOW ISN'T SUPPOSED TO WORK YET

# Initialize empty DataFrame to hold simulated prices for each simulation, and declare empty lists for the individual stock prices.
simulated_price_df = pd.DataFrame()
portfolio_cumulative_returns = pd.DataFrame()
simulated_agg_prices = []
simulated_spy_prices = []
simulated_spy_prices = []
simulated_spy_prices = []
simulated_spy_prices = []
simulated_spy_prices = []
simulated_spy_prices = []
simulated_spy_prices = []
simulated_spy_prices = []

# Run the simulation of projecting stock prices for the next trading year, `1000` times
for n in range(number_simulations):

    # Initialize the simulated prices list with the last closing price of `JNJ` and `MU`
    simulated_agg_prices = [agg_last_price]
    simulated_spy_prices = [spy_last_price]
    
    # Simulate the returns for 252 days
    for i in range(number_records):
        
        # Calculate the simulated price using the last price within the list
        simulated_agg_price = simulated_agg_prices[-1] * (1 + np.random.normal(avg_daily_return_agg, std_dev_daily_return_agg))
        simulated_spy_price = simulated_spy_prices[-1] * (1 + np.random.normal(avg_daily_return_spy, std_dev_daily_return_spy))
        
        # Append the simulated price to the list
        simulated_agg_prices.append(simulated_agg_price)
        simulated_spy_prices.append(simulated_spy_price)
    
    # Append the simulated prices of each simulation to DataFrame
    simulated_price_df["SPY prices"] = pd.Series(simulated_spy_prices)
    simulated_price_df["AGG prices"] = pd.Series(simulated_agg_prices)
    
    
    # Calculate the daily returns of simulated prices
    simulated_daily_returns = simulated_price_df.pct_change()
    
simulated_daily_returns.head()